The columns from left to right are name, hr, sigma_hr, T90 (or T50), sigma_T90

I suggest you work with T90.

Also note that if you have upgraded your sklearn, you will need to upgrade your codes to 

as GMM is now GaussianMixtureModels etc.

You can do two tests

1) without any outlier rejection

2) rejecting datapoints with large errors (with  rejecting values with error/value > 30% 

you can also try using Extreme Deconvolution (which accounts for errors while using GMM)

In [12]:
import pandas as pd
import numpy as np
import math
from sklearn.mixture import GaussianMixture
from matplotlib import pyplot as plt
#from astroML.plotting.tools import draw_ellipse
import itertools
from scipy import linalg
import matplotlib as mpl
from matplotlib.colors import LogNorm
from sklearn import mixture
# from xdgmm import XDGMM
from astroML.density_estimation import XDGMM

In [2]:
arr = np.genfromtxt("hrdata_10yr_T90_Final_w_err copy.txt", dtype=float,delimiter = "\t")
print(arr)

[[           nan 1.63549331e+00 1.61213324e+00 5.37599993e+00
  2.36019993e+00]
 [           nan 1.00478042e+00 2.31759034e+00 4.01920013e+01
  1.14487004e+00]
 [           nan 9.21419840e-01 1.96723148e-02 5.96487007e+01
  1.12757998e+01]
 ...
 [           nan 2.68814913e-01 1.90098389e-04 3.81440010e+01
  1.14487004e+00]
 [           nan 4.71753218e-01 1.86328835e-02 1.10080004e+01
  1.27999997e+00]
 [           nan 7.77065105e-01 4.92472397e-02 9.01119995e+01
  3.20766997e+00]]


In [3]:
t90=[]
hr=[]
dt90=[]
dhr=[]

for i in range(len(arr)):
    if arr[i][3] > 0 and arr[i][1] > 0:
        t90.append(math.log(arr[i][3]))
        hr.append(math.log(arr[i][1]))
        dhr.append(arr[i][1] / arr[i][2])
        dt90.append((arr[i][3]/arr[i][4]))

In [4]:
# stack the data 
X = np.vstack([t90, hr]).T
X = X[~np.isnan(X).any(axis=1)]
X = X[np.isfinite(X).any(axis=1)]

# stack data error
Xerr = np.zeros(X.shape + X.shape[-1:])
diag = np.arange(X.shape[-1])
Xerr[:, diag, diag] = np.vstack([dt90, dhr]).T

print X

[[ 1.68194459  0.49194448]
 [ 3.693668    0.00476903]
 [ 4.08847237 -0.08183949]
 ...
 [ 3.6413685  -1.31373219]
 [ 2.39862232 -0.75129927]
 [ 4.50105334 -0.25223114]]


In [5]:
# Define the range of component numbers, and get ready to compute the BIC for each one:
param_range = np.arange(1,6)
xdgmm = XDGMM()
bic, optimal_n_comp, lowest_bic = xdgmm.bic_test(X, Xerr, param_range)

N = 1 , BIC = 23093.121463997013
N = 2 , BIC = 22979.55883233428
N = 3 , BIC = 23024.3292499457
N = 4 , BIC = 23043.14823485075


/usr/local/lib/python2.7/dist-packages/sklearn/mixture/base.py:237: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


N = 5 , BIC = 23053.35524038397


In [ ]:
plt.plot(param_range, bic, ':k', label='BIC', color = 'red', marker='^',lw=2)
plt.legend(loc=1)
plt.xlabel('Number of components',size=20 )
plt.ylabel('AIC/BIC', size=20)
plt.show()